In [7]:
from PIL import Image
import json
import glob
import pandas as pd
 
# image_name = "image_00026"
json_folder = "redjson"
label_folder = "redlabels"
split_folder = "redlabelsplit"
json_split_folder = "redlabelsplitjson"

for name in glob.glob(json_folder + "/*.json"):
#     print(name)
    image_path = label_folder + "/" + name[-16:-5] + ".png"
    json_path = name
    out_path = split_folder + "/" + name[-16:-5]

    out_json_path = json_split_folder + '/'
#     print(image_path)
#     print(json_path)
    

    img = Image.open(image_path).convert("RGB")
    width = img.width
    height = img.height
    px = img.load()

    f = open(json_path)
    labels = json.load(f)
    materials = []
    for rows in labels:
        for columns in labels[rows]:
            material = labels[rows][columns]
            if material not in materials:
                materials.append(material)
    materials.remove("Material93")

    #Here's where I create another json file -> for each image, I need a corresponding json that maps to sub image given a label.
    #Therefore, there will be a json file called image_xxxxx_segmentation.json that maps a structure to a split image (e.g. image_xxxxx_x.png)
    labels_json_list = {}

    for index in range(len(materials)):
        structure = materials[index]
        image = Image.new("RGBA", (width, height))
#         print(structure)
        for rows in labels:
            for columns in labels[rows]:
                if labels[rows][columns] == structure:
                    pixel = img.getpixel((int(rows), int(columns)))
                    image.putpixel((int(rows), int(columns)), pixel)
                else:
                    image.putpixel((int(rows), int(columns)), (0, 0, 0, 0))

        image.save(out_path + "_" + str(index) + "_" + ".png")

        #Append to a the dict: label->image
        labels_json_list[structure] = name[-16:-5] + '_' + str(index) + "_" + ".png"
        #print(str(structure), name[-16:-5] + '_' + str(index) + "_" + ".png")
        
        

    #Save the new json file to a folder. 
    #First convert to a DF
    print(labels_json_list)
    splits_json_df = pd.Series(labels_json_list)
    splits_json_df.to_json(out_json_path + name[-16:-5] + '_splits.json')
    #print(out_json_path + name[-16:-5] + '_splits.json')


{'Unknown Tissue': 'image_00006_0_.png', 'cerebral peduncle left': 'image_00006_1_.png', 'inferior cerebellar peduncle right': 'image_00006_2_.png', 'inferior olive principal nucleus right': 'image_00006_3_.png', 'cuneate fasciculus left': 'image_00006_4_.png', 'spinothalamic tract right': 'image_00006_5_.png', 'spinal trigeminal tract right': 'image_00006_6_.png', 'external cuneate nucleus right': 'image_00006_7_.png', 'spinal trigeminal nucleus interpolaris right': 'image_00006_8_.png', 'mesencephalic trigeminal tract left': 'image_00006_9_.png', 'vestibular nucleus right': 'image_00006_10_.png', 'medial lemniscus right': 'image_00006_11_.png', 'medial longitudinal fasciulus right': 'image_00006_12_.png', 'medial lemniscus left': 'image_00006_13_.png', 'corticospinal tract left': 'image_00006_14_.png', 'medial longitudinal fasciulus left': 'image_00006_15_.png', 'abducens nucleus right': 'image_00006_16_.png', 'inferior olive principal nucleus left': 'image_00006_17_.png', 'vestibula

ValueError: If using all scalar values, you must pass an index

In [ ]:
#Duncan adding json files for the label overlay...